# Part 1: Scraping the Data

In [41]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
#!conda install -c conda-forge folium=0.5.0 --yes
import matplotlib.cm as cm
import matplotlib.colors as colors
from pandas.io.json import json_normalize
import folium
from sklearn.cluster import KMeans

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(url,'utf-8')
soup = BeautifulSoup(r.content, 'html5lib')  

In [3]:
table = soup.find('table',attrs = {'class': 'wikitable sortable'})

In [4]:
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]

In [5]:
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [6]:
df.shape

(180, 3)

**Processing the data**

In [7]:
df.drop(df[df['Borough']=='Not assigned'].index, axis = 0, inplace = True)

In [8]:
df.sort_values('Postal code',inplace = True)

In [9]:
df.head(11)

,Postal code,Borough,Neighborhood
9,M1B,Scarborough,Malvern / Rouge
18,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
27,M1E,Scarborough,Guildwood / Morningside / West Hill
36,M1G,Scarborough,Woburn
45,M1H,Scarborough,Cedarbrae
54,M1J,Scarborough,Scarborough Village
63,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park
72,M1L,Scarborough,Golden Mile / Clairlea / Oakridge
81,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West
90,M1N,Scarborough,Birch Cliff / Cliffside West


In [10]:
df.shape

(103, 3)

# Part 2: Geolocation

In [11]:
geo_loc = pd.read_csv("http://cocl.us/Geospatial_data")

In [12]:
geo_loc.shape

(103, 3)

In [13]:
final = pd.merge(df,geo_loc, left_on = 'Postal code', right_on = 'Postal Code').drop('Postal Code',axis = 1)

In [14]:
final.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Part 3: Clustering

**Lets visualize the map**

In [15]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [16]:
# create map of Toronto using latitude and longitude values
toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(final['Latitude'], final['Longitude'], final['Borough'], final['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto)  
    
toronto

## I'm going to focus on Scarborough

In [17]:
scar_data = final[final['Borough'] == 'Scarborough'].reset_index(drop=True)

In [18]:
scar_data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [19]:
scar_data.shape

(17, 5)

In [20]:
address = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [21]:
# create map of Scarborough using latitude and longitude values
scar_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scar_data['Latitude'], scar_data['Longitude'], scar_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#8ff7ab',
        fill_opacity=0.7,
        parse_html=False).add_to(scar_map)  
    
scar_map

In [22]:
CLIENT_ID = 'XDUJ01EV2EADGKO0M0GQCQPO3CG3FNDJR4TA32UNHRLDQ0XV' # your Foursquare ID
CLIENT_SECRET = 'UNCLFC4DSC4BPHUNCGEVOSAEOESF5SB25AKTY3UR1YG33WZL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XDUJ01EV2EADGKO0M0GQCQPO3CG3FNDJR4TA32UNHRLDQ0XV
CLIENT_SECRET:UNCLFC4DSC4BPHUNCGEVOSAEOESF5SB25AKTY3UR1YG33WZL


#### Let's explore the first neighborhood in our dataframe.

In [23]:
neigh_latitude = scar_data.loc[0, 'Latitude'] # neighborhood latitude value
neigh_longitude = scar_data.loc[0, 'Longitude'] # neighborhood longitude value

neigh_name = scar_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neigh_name, neigh_latitude, neigh_longitude))

Latitude and longitude values of Malvern / Rouge are 43.806686299999996, -79.19435340000001.


#### Let's see the top 50 venues in neighborhood

In [24]:
# type your answer here
LIMIT = 50
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neigh_latitude, 
    neigh_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()

In [25]:
#getting categories of each venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [26]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy’s,Fast Food Restaurant,43.807448,-79.199056


In [27]:
nearby_venues.shape[0]

1

#### Creating a function to do the same for all neighborhoods

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
scar_venues = getNearbyVenues(names=scar_data['Neighborhood'],
                                   latitudes=scar_data['Latitude'],
                                   longitudes=scar_data['Longitude']
                                  )

Malvern / Rouge
Rouge Hill / Port Union / Highland Creek
Guildwood / Morningside / West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park / Ionview / East Birchmount Park
Golden Mile / Clairlea / Oakridge
Cliffside / Cliffcrest / Scarborough Village West
Birch Cliff / Cliffside West
Dorset Park / Wexford Heights / Scarborough Town Centre
Wexford / Maryvale
Agincourt
Clarks Corners / Tam O'Shanter / Sullivan
Milliken / Agincourt North / Steeles East / L'Amoreaux East
Steeles West / L'Amoreaux West
Upper Rouge


In [30]:
print('There are {} uniques categories.'.format(len(scar_venues['Venue Category'].unique())))

There are 56 uniques categories.


In [31]:
scar_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Malvern / Rouge,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,Guildwood / Morningside / West Hill,43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,Guildwood / Morningside / West Hill,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


## Processing the data and Analyzing it

In [32]:
# one hot encoding
scar_onehot = pd.get_dummies(scar_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scar_onehot['Neighborhood'] = scar_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scar_onehot.columns[-1]] + list(scar_onehot.columns[:-1])
scar_onehot = scar_onehot[fixed_columns]

print(scar_onehot.shape)
scar_onehot.head()

(94, 57)


,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,...,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Supermarket,Thai Restaurant,Train Station,Vietnamese Restaurant
0,Malvern / Rouge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Rouge Hill / Port Union / Highland Creek,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Rouge Hill / Port Union / Highland Creek,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Guildwood / Morningside / West Hill,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Guildwood / Morningside / West Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
scar_grouped = scar_onehot.groupby('Neighborhood').mean().reset_index()
scar_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,...,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Supermarket,Thai Restaurant,Train Station,Vietnamese Restaurant
0,Agincourt,0.0,0.000,0.000,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,...,0.000000,0.000000,0.000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000
1,Birch Cliff / Cliffside West,0.0,0.000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000,0.25,0.000,0.000000,0.000000,0.000000,0.000000,0.000
2,Cedarbrae,0.0,0.125,0.000,0.125000,0.125000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000,0.00,0.000,0.000000,0.000000,0.125000,0.000000,0.000
3,Clarks Corners / Tam O'Shanter / Sullivan,0.0,0.000,0.000,0.000000,0.071429,0.0,0.000000,0.000000,0.000000,...,0.071429,0.000000,0.000,0.00,0.000,0.000000,0.000000,0.071429,0.000000,0.000
4,Cliffside / Cliffcrest / Scarborough Village West,0.5,0.000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000
5,Dorset Park / Wexford Heights / Scarborough To...,0.0,0.000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.200
6,Golden Mile / Clairlea / Oakridge,0.0,0.000,0.000,0.222222,0.000000,0.0,0.000000,0.222222,0.000000,...,0.000000,0.000000,0.000,0.00,0.000,0.111111,0.000000,0.000000,0.000000,0.000
7,Guildwood / Morningside / West Hill,0.0,0.000,0.000,0.000000,0.142857,0.0,0.142857,0.000000,0.000000,...,0.142857,0.000000,0.000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000
8,Kennedy Park / Ionview / East Birchmount Park,0.0,0.000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.166667,...,0.000000,0.000000,0.000,0.00,0.000,0.000000,0.000000,0.000000,0.166667,0.000
9,Malvern / Rouge,0.0,0.000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000


#### Let's print each neighborhood along with the top 5 most common venues

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scar_grouped['Neighborhood']

for ind in np.arange(scar_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scar_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Latin American Restaurant,Lounge,Chinese Restaurant,Vietnamese Restaurant,Department Store,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station
1,Birch Cliff / Cliffside West,College Stadium,General Entertainment,Skating Rink,Café,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,Gas Station,Furniture / Home Store,Fried Chicken Joint
2,Cedarbrae,Thai Restaurant,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Vietnamese Restaurant,Department Store
3,Clarks Corners / Tam O'Shanter / Sullivan,Pizza Place,Noodle House,Fast Food Restaurant,Chinese Restaurant,Pharmacy,Italian Restaurant,Intersection,Rental Car Location,Bank,Fried Chicken Joint
4,Cliffside / Cliffcrest / Scarborough Village West,American Restaurant,Motel,Hobby Shop,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
5,Dorset Park / Wexford Heights / Scarborough To...,Indian Restaurant,Vietnamese Restaurant,Chinese Restaurant,Pet Store,Convenience Store,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store
6,Golden Mile / Clairlea / Oakridge,Bakery,Bus Line,Ice Cream Shop,Metro Station,Park,Soccer Field,Intersection,Fast Food Restaurant,Discount Store,Electronics Store
7,Guildwood / Morningside / West Hill,Bank,Intersection,Breakfast Spot,Rental Car Location,Electronics Store,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Department Store,Grocery Store
8,Kennedy Park / Ionview / East Birchmount Park,Hobby Shop,Department Store,Bus Station,Discount Store,Coffee Shop,Train Station,History Museum,Hakka Restaurant,Grocery Store,General Entertainment
9,Malvern / Rouge,Fast Food Restaurant,Vietnamese Restaurant,Train Station,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint


## Clustering

In [36]:
# set number of clusters
kclusters = 5

scar_grouped_clustering = scar_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scar_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 3, 1, 1, 1, 1, 2], dtype=int32)

In [37]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scar_merged = scar_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scar_merged = scar_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

print(scar_merged.shape)
scar_merged.head()

(17, 16)


,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353,2.0,Fast Food Restaurant,Vietnamese Restaurant,Train Station,History Museum,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,4.0,History Museum,Bar,Vietnamese Restaurant,Convenience Store,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711,1.0,Bank,Intersection,Breakfast Spot,Rental Car Location,Electronics Store,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Department Store,Grocery Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Insurance Office,Korean Restaurant,Vietnamese Restaurant,Convenience Store,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Thai Restaurant,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Vietnamese Restaurant,Department Store


In [38]:
scar_merged.dropna(inplace = True)
scar_merged.shape

(16, 16)

In [39]:
scar_merged['Cluster Labels'] = scar_merged['Cluster Labels'].astype('int8')

In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scar_merged['Latitude'], scar_merged['Longitude'], scar_merged['Neighborhood'], scar_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters